In [1]:
# https://www.geeksforgeeks.org/scrape-linkedin-using-selenium-and-beautiful-soup-in-python/

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import datetime
import time
import json

## Logging in to LinkedIn

In [3]:
# Creating a webdriver instance
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\Joyce\AppData\Local\Temp\ipykernel_16024\2930654737.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [4]:
# Opening linkedIn's login page
driver.get("https://linkedin.com/uas/login")
# waiting for the page to load
time.sleep(2)

In [5]:
# entering username
username = driver.find_element(By.ID, "username")
# Enter Your Email Address
username.send_keys("hzzbsch@gmail.com")

In [6]:
# entering password
pword = driver.find_element(By.ID, "password")
# Enter Your Password
pword.send_keys("cs5412")

In [7]:
# Clicking on the log in button
# Format (syntax) of writing XPath --> //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()

## Extracting Job Search Data

In [8]:
# Opening job's page
# paste the URL of job's page here
job_url = "https://www.linkedin.com/jobs/"
# this will open the link
driver.get(job_url)
time.sleep(2)

In [9]:
# entering search keywords
search = driver.find_element(By.XPATH, "//input[starts-with(@id, 'jobs-search-box-keyword-id-ember')]")
search.send_keys("software engineer")
time.sleep(2)
search.send_keys(Keys.RETURN)
time.sleep(2)

In [10]:
# scroll
def scroll():
    print("start scrolling", end = ' ')
    left_panel = driver.find_element(By.CLASS_NAME, "jobs-search-results-list")
    time.sleep(1)
    verical_ordinate = 100
    for i in range(0, 10):
        print(".", end = ' ')
        driver.execute_script("arguments[0].scrollTop = arguments[1]", left_panel, verical_ordinate)
        verical_ordinate += 1000
        time.sleep(1)
    print("done scrolling")

In [11]:
# process jobs on one page
def process_page():
    job_src = driver.page_source
    soup = BeautifulSoup(job_src, 'lxml')
    # get the list of job titles
    jobs_html = soup.find_all('a', {'class': 'job-card-list__title'})

    # loop through jobs on this page
    for title in jobs_html:
        try:
            job_data = {}
            print("processing position: " + title.text.strip())
            job_data["position"] = title.text.strip()

            # click on each job to get detailed information
            driver.find_element(By.LINK_TEXT, title.text.strip()).click()
            time.sleep(2)
            detail_src = driver.page_source
            soup = BeautifulSoup(detail_src, 'lxml')
            detail = soup.find('section', {'class': 'scaffold-layout__detail'})

            # get job id
            url = driver.current_url
            index_start = url.find('currentJobId=') + 13
            index_end = url.find('&keywords=')
            info_id = url[index_start:index_end] if (index_start != -1 and index_end != -1) else ""
            job_data["job_id"] = info_id

            # get company name
            info_company = soup.find('span', {'class': 'jobs-unified-top-card__company-name'})
            job_data["company"] = info_company.text.strip() if info_company is not None else ""

            # get date posted
            info_date = soup.find('span', {'class': 'jobs-unified-top-card__posted-date'})
            job_data["date"] = info_date.text.strip() if info_date is not None else ""

            # get type and level
            info_type = soup.find('li', {'class': 'jobs-unified-top-card__job-insight'})
            job_data["type"] = info_type.text.strip() if info_type is not None else ""

            # get description
            info_description = soup.find('div', {'class': 'jobs-description-content'})
            job_data["description"] = info_description.text.strip() if info_description is not None else ""
            data.append(job_data)
        except:
            print("something wrong here :(")

In [12]:
# loop through the pages
page_lst = driver.find_element(By.CLASS_NAME, "artdeco-pagination__pages")
pages = page_lst.find_elements(By.TAG_NAME, "li")
start = int(pages[0].text)
end = int(pages[-1].text)
data = []
time.sleep(2)

for page in range(start, 2):
    driver.find_element(By.XPATH, "//button[@aria-label='Page " + str(page) + "']").click()
    time.sleep(2)
    print("currently on page: " + str(page))
    try:
        scroll()
        process_page()
        print("================================================")
    except:
        print("something wrong here :(")

currently on page: 1
start scrolling . . . . . . . . . . done scrolling
processing position: CNC Programmer
processing position: Software Engineer
processing position: Software Engineer III - (E3)
processing position: PHP/Java Developer
processing position: Mobile Engineer
processing position: Senior Full Stack Engineer
processing position: Senior Software Engineer
processing position: Intern, Software Engineer
processing position: Intern, Software Engineer
processing position: Azure Data Engineer
processing position: Senior Software Engineer
processing position: Application Developer
processing position: Web Developer
processing position: Intern, Software Engineer
processing position: Software Engineering Intern
processing position: React Native/Full Stack Developer
processing position: Sr. Full Stack Engineer
processing position: Senior Mobile Developer
processing position: Senior Software Engineer, Frontend
processing position: Senior System Engineer


In [13]:
json_object = json.dumps(data)
with open(("LinkedIn" + datetime.today().strftime('%Y%m%d%H%M') + ".json"), "w") as outfile:
    outfile.write(json_object)